# Procesamiento de datasets
En este notebook están agrupados los procedimientos por los cuales se importaron y procesaron los datos originalmente provistos en el set `Womens_Clothing_Reviews.csv` para generar un set de datos con los atributos relevantes y las reviews lemmatizadas y stemmatizadas como columnas aparte. Así se obtiene el archivo: `dataset_clothes_clean.csv`.

De esta forma se ahorra tiempo en la ejecución del notebook que resulve la consigna pedida.

### Importar las librerías necesarias

In [1]:
import pandas as pd
import numpy as np

# Garbage collector para optimizar recursos
import gc

In [2]:
# Previamente
## python -m spacy download en_core_web_md

import spacy # https://spacy.io/usage/models
nlp = spacy.load('en_core_web_md')

#Stop Words de en_core_news_md
from spacy.lang.en.stop_words import STOP_WORDS
stopwords_spacy = list(STOP_WORDS)

In [3]:
import nltk

#Stop Words de nltk
from nltk.corpus import stopwords
stopwords_nltk = set(stopwords.words('english'))

import re
from nltk.tokenize import RegexpTokenizer

In [4]:
# Clase para personalizar las impresiones de consola
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

### Importar el dataset original

In [5]:
data = pd.read_csv("data/Womens_Clothing_Reviews.csv")
print("- Cantidad de filas del set:" + color.YELLOW, data.shape[0],color.END)
print("- Cantidad de atributos (columnas) del dataset:" + color.CYAN, data.shape[1],color.END)

- Cantidad de filas del set: 23486 
- Cantidad de atributos (columnas) del dataset: 11 


### Filtrado de columnas
Nos quedamos con las columnas del dataset que son relevantes para predecir si una prenda será recomendada o no. Por eso se descartan las columnas de `Positive Feedback Count`, `Division Name` y `Department Name`. La de clase nos puede servir para relacionar palabras y la de edad podría tener alguna influencia en el resultado final. El id de la prenda, si bien muy probablemente no sirva para la predicción, sí sirve para el análisis exploratorio y entender a qué prendas en particular se hace referencia.

In [6]:
# Filtramos columnas y las renombramos para mayor facilidad de uso
data = data[['Class Name','Clothing ID','Age','Title','Review Text','Rating','Recommended IND']]
mapper = {'Clothing ID':'clothing_id', 'Age':'usr_age', 'Title':'review_title', 'Review Text':'review_body', 'Rating':'rating',
       'Recommended IND':'recommended', 'Class Name':'class_name'}
data.rename(columns=mapper,inplace=True)
data.head()

,class_name,clothing_id,usr_age,review_title,review_body,rating,recommended
0,Intimates,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1
1,Dresses,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1
2,Dresses,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0
3,Pants,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1
4,Blouses,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1


### Limpieza de datos

In [7]:
# Tratamiento de nulos
df  = data.copy()
df.fillna(value={'review_title':"-"}, inplace=True)
df = df.dropna()
df.reset_index(drop=True, inplace=True)

# Porcentaje de nulos por columna
df.isna().sum()*100/df.shape[0]

class_name      0.0
clothing_id     0.0
usr_age         0.0
review_title    0.0
review_body     0.0
rating          0.0
recommended     0.0
dtype: float64

In [8]:
#Constante de signos de puntuación
import string
puntua = string.punctuation + '#...'
excluded_pos = ['SCONJ','CCONJ','NUM','PUNCT','PRON','DET','ADP','AUX','X']

In [9]:
#Función para limpieza de datos con lemmatizer
def text_data_lemma(sentence):
    doc = nlp(sentence)
    
    clean_tokens = []
    for token in doc:
        if (token.pos_ not in excluded_pos and str(token) not in stopwords_spacy and len(token.text)>2): 
            temp = token.lemma_.strip()
            clean_tokens.append(temp.lower())
    
    return clean_tokens

In [10]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

#Función para limpieza de datos con stemmer
def text_data_stem(sentence):
    doc = nlp(sentence)
    
    clean_tokens = []
    for token in doc:
        if (token.pos_ not in excluded_pos and str(token) not in stopwords_spacy and len(token.text)>2): 
            temp = stemmer.stem(token.text).strip()
            clean_tokens.append(temp.lower())
    
    return clean_tokens

#### Lemmatización (10 min aprox)

In [11]:
# Limpiamos todas las reviews con lemmatizer
reviews_lemma = []
for i in df.index:
    rev = text_data_lemma(df.review_title.iloc[i] + ' ' + df.review_body.iloc[i])
    reviews_lemma.append(" ".join(rev))
reviews_lemma[:5]

['absolutely wonderful silky sexy comfortable',
 'love dress sooo pretty happen find store glad order online petite buy petite love length me- hit little knee definitely true midi truly petite',
 'major design flaw high hope dress want work initially order petite small usual size find outrageously small small fact zip reorder petite medium overall half comfortable fit nicely half tight layer somewhat cheap net layer imo major design flaw net layer sew directly zipper',
 'favorite buy love love love jumpsuit fun flirty fabulous time wear great compliment',
 'flattering shirt shirt flattering adjustable tie perfect length wear legging sleeveless pair cardigan love shirt']

#### Stemmización (10 min aprox)

In [12]:
# Limpiamos todas las reviews con stemmizer
reviews_stem = []
for i in df.index:
    rev = text_data_stem(df.review_title.iloc[i] + ' ' + df.review_body.iloc[i])
    reviews_stem.append(" ".join(rev))
reviews_stem[:5]

['absolut wonder silki sexi comfort',
 'love dress sooo pretti happen find store glad order onlin petit bought petit love length me- hit littl knee definit true midi truli petit',
 'major design flaw high hope dress want work initi order petit small usual size found outrag small small fact zip reorder petit medium overal half comfort fit nice half tight layer somewhat cheap net layer imo major design flaw net layer sewn directli zipper',
 'favorit buy love love love jumpsuit fun flirti fabul time wear great compliment',
 'flatter shirt shirt flatter adjust tie perfect length wear leg sleeveless pair cardigan love shirt']

### Guardar nuevos datos en achivo .csv

In [13]:
# Agregamos columna al dataset
df['revs_lemma'] = reviews_lemma
df['revs_stem'] = reviews_stem
df.head()

,class_name,clothing_id,usr_age,review_title,review_body,rating,recommended,revs_lemma,revs_stem
0,Intimates,767,33,-,Absolutely wonderful - silky and sexy and comf...,4,1,absolutely wonderful silky sexy comfortable,absolut wonder silki sexi comfort
1,Dresses,1080,34,-,Love this dress! it's sooo pretty. i happene...,5,1,love dress sooo pretty happen find store glad ...,love dress sooo pretti happen find store glad ...
2,Dresses,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,major design flaw high hope dress want work in...,major design flaw high hope dress want work in...
3,Pants,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,favorite buy love love love jumpsuit fun flirt...,favorit buy love love love jumpsuit fun flirti...
4,Blouses,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,flattering shirt shirt flattering adjustable t...,flatter shirt shirt flatter adjust tie perfect...


In [14]:
# Guardamos dataset lemmatizado
df.to_csv(path_or_buf='data/dataset_clothes_clean.csv', index=False) # Para que no guarde el índice como otra columna